<a href="https://colab.research.google.com/github/EmilyCarroll-del/Michael-J-Fox-Foundation-FOG-in-PD/blob/main/MJF_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/'
%ls

/content/drive/MyDrive/Colab Notebooks
'Copy of defog_start_hesitation.csv'          defog_turn_draft.csv
'Copy of defog_start_hesitation_test.csv'     defog_turn_test_draft.csv
'Copy of defog_turn.csv'                      defog_walk_draft.csv
'Copy of defog_turn_test.csv'                 defog_walk_test_draft.csv
'Copy of defog_walking.csv'                   MJFF-FOG-Prediction-PD.ipynb
'Copy of defog_walking_test.csv'              tdcsfog_sh_draft.csv
'Copy of tdcsfog_start_hesitation.csv'        tdcsfog_sh_test_draft.csv
'Copy of tdcsfog_start_hesitation_test.csv'   tdcsfog_turn_draft.csv
'Copy of tdcsfog_turn.csv'                    tdcsfog_turn_test_draft.csv
'Copy of tdcsfog_turn_test.csv'               tdcsfog_walk_draft.csv
'Copy of tdcsfog_walking.csv'                 tdcsfog_walk_test_draft.csv
'Copy of tdcsfog_walking_test.csv'            Test/
 defog_sh_draft.csv                           tlvmc-parkinsons-freezing-gait-prediction/
 defog_sh_test_draft.csv


In [ ]:
!pip install tsflex seglearn tensorflow Keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from tsflex.features import FeatureCollection, MultipleFeatureDescriptors
from tsflex.features.integrations import seglearn_feature_dict_wrapper
from seglearn.feature_functions import base_features, emg_features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
def process_directory(directory):
    dfs = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)

        # Check if the file is a CSV and process it
        if file_name.endswith('.csv') and os.path.isfile(file_path):
            df = pd.read_csv(file_path) #read csv files
            df['source_directory'] = os.path.basename(directory)  # Add a column to identify the source directory
            df['csv_name'] = os.path.basename(file_name)
            dfs.append(df)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

In [ ]:
defog = process_directory('/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/defog')
tdcsfog = process_directory('/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog')
#notype = process_directory('/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/notype')

In [ ]:
folder_path = "/content/drive/My Drive/Colab Notebooks/Test"
file_name = "Copy of 3e6987cb2d.csv"
file_path = os.path.join(folder_path, file_name)
defog_test = pd.read_csv(file_path)
defog_test.shape

(300288, 9)

In [ ]:
folder_path = "/content/drive/My Drive/Colab Notebooks/Test"
file_name = "Copy of 3ba3590a08.csv"
file_path = os.path.join(folder_path, file_name)
tdcsfog_test = pd.read_csv(file_path)
tdcsfog_test.shape

(46817, 7)

In [ ]:
#filter the accelerometer data - defog training

from scipy.signal import butter, filtfilt

fs = 100  # Sampling frequency in Hz

# Define filter parameters
lowcut = 0.25
highcut = 10.0
order = 4  # Filter order

# Create a Butterworth bandpass filter
nyquist = 0.5 * fs
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(order, [low, high], btype="band")

# Apply the filter to each accelerometer axis
defog["AccV_filtered"] = filtfilt(b, a, defog["AccV"])
defog["AccML_filtered"] = filtfilt(b, a, defog["AccML"])
defog["AccAP_filtered"] = filtfilt(b, a, defog["AccAP"])

print(defog.head())

   Time  AccV     AccML  AccAP  StartHesitation  Turn  Walking  Valid   Task  \
0     0  -1.0  0.044129  -0.25                0     0        0  False  False   
1     1  -1.0  0.034431  -0.25                0     0        0  False  False   
2     2  -1.0  0.031250  -0.25                0     0        0  False  False   
3     3  -1.0  0.031250  -0.25                0     0        0  False  False   
4     4  -1.0  0.031250  -0.25                0     0        0  False  False   

  source_directory        csv_name  AccV_filtered  AccML_filtered  \
0            defog  02ea782681.csv      -0.000051        0.000237   
1            defog  02ea782681.csv      -0.000051       -0.004508   
2            defog  02ea782681.csv      -0.000050       -0.008581   
3            defog  02ea782681.csv      -0.000050       -0.011507   
4            defog  02ea782681.csv      -0.000050       -0.013125   

   AccAP_filtered  
0       -0.000733  
1       -0.000764  
2       -0.000796  
3       -0.000829  
4   

In [ ]:
#filter the accelerometer data - tdcsfog training

from scipy.signal import butter, filtfilt

fs = 128  # Sampling frequency in Hz

# Define filter parameters
lowcut = 0.25
highcut = 10.0
order = 4  # Filter order

# Create a Butterworth bandpass filter
nyquist = 0.5 * fs
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(order, [low, high], btype="band")

# Apply the filter to each accelerometer axis
tdcsfog["AccV_filtered"] = filtfilt(b, a, tdcsfog["AccV"])
tdcsfog["AccML_filtered"] = filtfilt(b, a, tdcsfog["AccML"])
tdcsfog["AccAP_filtered"] = filtfilt(b, a, tdcsfog["AccAP"])

print(tdcsfog.head())

   Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
0     0 -9.533939  0.566322 -1.413525                0     0        0   
1     1 -9.536140  0.564137 -1.440621                0     0        0   
2     2 -9.529345  0.561765 -1.429332                0     0        0   
3     3 -9.531239  0.564227 -1.415490                0     0        0   
4     4 -9.540825  0.561854 -1.429471                0     0        0   

  source_directory        csv_name  AccV_filtered  AccML_filtered  \
0          tdcsfog  003f117e14.csv       0.001085        0.000978   
1          tdcsfog  003f117e14.csv       0.000986       -0.000828   
2          tdcsfog  003f117e14.csv       0.000908       -0.002730   
3          tdcsfog  003f117e14.csv       0.000866       -0.004786   
4          tdcsfog  003f117e14.csv       0.000860       -0.006998   

   AccAP_filtered  
0        0.006895  
1        0.003350  
2        0.000218  
3       -0.002185  
4       -0.003692  


In [ ]:
#filtering for defog_test
fs = 100
lowcut = 0.25
highcut = 10.0
order = 4

nyquist = 0.5 * fs
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(order, [low, high], btype="band")

defog_test["AccV_filtered"] = filtfilt(b, a, defog_test["AccV"])
defog_test["AccML_filtered"] = filtfilt(b, a, defog_test["AccML"])
defog_test["AccAP_filtered"] = filtfilt(b, a, defog_test["AccAP"])

In [ ]:
#filtering for tdcsfog_test
fs = 128
lowcut = 0.25
highcut = 10.0
order = 4

nyquist = 0.5 * fs
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(order, [low, high], btype="band")

tdcsfog_test["AccV_filtered"] = filtfilt(b, a, tdcsfog_test["AccV"])
tdcsfog_test["AccML_filtered"] = filtfilt(b, a, tdcsfog_test["AccML"])
tdcsfog_test["AccAP_filtered"] = filtfilt(b, a, tdcsfog_test["AccAP"])

In [ ]:
#Windowing for defog
def create_overlapping_windows(data, window_size, overlap, columns=['AccV_filtered', 'AccML_filtered', 'AccAP_filtered']):
    """
    Parameters:
        data (pd.DataFrame): Input DataFrame.
        window_size (int): Number of samples in each window.
        overlap (int): Number of overlapping samples between consecutive windows.
        columns (list): Columns to extract for windows.

    Returns:
        np.ndarray: 3D tensor of shape (NSegments, window_size, len(columns)).
    """
    step = window_size - overlap
    windows = []
    for start in range(0, len(data) - window_size + 1, step):
        end = start + window_size
        windows.append(data.iloc[start:end][columns].values)

    return np.array(windows)

# Parameters
window_size = 300  # 3 seconds * 100 Hz
overlap = 100      # 1 second * 100 Hz

#defog_tensor = create_overlapping_windows(defog, window_size, overlap)

#print(f"Tensor shape: {defog_tensor.shape}")  # Should be (NSegments, 300, 3)

In [ ]:
#Windowing for tdcsfog
window_size = 384  # 3 seconds * 128 Hz
overlap = 128      # 1 second * 128 Hz

#tdcsfog_tensor = create_overlapping_windows(tdcsfog, window_size, overlap)

#print(f"Tensor shape: {tdcsfog_tensor.shape}")  # Should be (NSegments, 300, 3)


In [ ]:
# Parameters
window_size = 300  # 3 seconds * 100 Hz
overlap = 100      # 1 second * 100 Hz

# Create overlapping windows for training and test data
X_train = create_overlapping_windows(defog, window_size, overlap)
X_test = create_overlapping_windows(defog_test, window_size, overlap)

In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(defog) - window_size + 1, step)
]

y_train = [
    max(defog['Turn'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_train = np.array(y_train).reshape(-1, 1)

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

X_train shape: (66126, 300, 3), y_train shape: (66126, 1)


In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(defog_test) - window_size + 1, step)
]

y_test = [
    max(defog_test['Turn'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_test = np.array(y_test).reshape(-1, 1)

# Verify shapes
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_test shape: (1500, 300, 3), y_test shape: (1500, 1)


In [ ]:
# Count label occurrences in y_train
unique, counts = np.unique(y_train, return_counts=True)
print("Training Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

# Count label occurrences in y_test
unique, counts = np.unique(y_test, return_counts=True)
print("\nTest Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

Training Label Distribution:
Label 0: 62086 samples
Label 1: 4040 samples

Test Label Distribution:
Label 0: 1261 samples
Label 1: 239 samples


In [ ]:
#training an LSTM model on defog
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=16,
    class_weight=class_weight_dict,
)

# Evaluate the model
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 1051s 253ms/step - Precision: 0.0622 - Recall: 0.4662 - accuracy: 0.5441 - loss: 0.6908 - val_Precision: 0.1593 - val_Recall: 1.0000 - val_accuracy: 0.1593 - val_loss: 0.6944
Epoch 2/5
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 1049s 240ms/step - Precision: 0.0602 - Recall: 0.3525 - accuracy: 0.6332 - loss: 0.6855 - val_Precision: 0.1593 - val_Recall: 1.0000 - val_accuracy: 0.1593 - val_loss: 0.7044
Epoch 3/5
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 1050s 242ms/step - Precision: 0.0610 - Recall: 0.6569 - accuracy: 0.3578 - loss: 0.6954 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.8407 - val_loss: 0.6862
Epoch 4/5
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 1003s 243ms/step - Precision: 0.0606 - Recall: 0.4844 - accuracy: 0.5036 - loss: 0.7005 - val_Precision: 0.1593 - val_Recall: 1.0000 - val_accuracy: 0.1593 - val_loss: 0.6978
Epoch 5/5
4133/4133 ━━━━━━━━━━━━━━━━━━━━ 1039s 242ms/step - Precision: 0.0582 - Recall: 0.4153 - accuracy: 0.5609 - loss: 0.6881

In [ ]:
from sklearn.metrics import roc_curve

y_pred_probs = model.predict(X_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

# Find optimal threshold (e.g., maximum TPR - FPR)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold}")

# Apply threshold to predictions
y_pred = (y_pred_probs > optimal_threshold).astype(int)

47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step
Optimal Threshold: 0.5036123991012573


In [ ]:
from sklearn.metrics import classification_report

# Predict binary labels
#y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
y_true = y_test.flatten()

print("Classification Report:")
print(classification_report(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      1261
           1       0.33      0.35      0.34       239

    accuracy                           0.78      1500
   macro avg       0.60      0.61      0.60      1500
weighted avg       0.79      0.78      0.79      1500



In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(defog) - window_size + 1, step)
]

y_train = [
    max(defog['Walking'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_train = np.array(y_train).reshape(-1, 1)

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

X_train shape: (66126, 300, 3), y_train shape: (66126, 1)


In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(defog_test) - window_size + 1, step)
]

y_test = [
    max(defog_test['Walking'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_test = np.array(y_test).reshape(-1, 1)

# Verify shapes
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_test shape: (1500, 300, 3), y_test shape: (1500, 1)


In [ ]:
# Count label occurrences in y_train
unique, counts = np.unique(y_train, return_counts=True)
print("Training Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

# Count label occurrences in y_test
unique, counts = np.unique(y_test, return_counts=True)
print("\nTest Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

Training Label Distribution:
Label 0: 65288 samples
Label 1: 838 samples

Test Label Distribution:
Label 0: 1486 samples
Label 1: 14 samples


In [ ]:
#training an LSTM model on defog - walking
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=32,
    class_weight=class_weight_dict
)


# Evaluate the model
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 589s 283ms/step - Precision: 0.0107 - Recall: 0.2255 - accuracy: 0.7630 - loss: 0.6845 - val_Precision: 0.0093 - val_Recall: 1.0000 - val_accuracy: 0.0093 - val_loss: 0.7347
Epoch 2/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 618s 281ms/step - Precision: 0.0142 - Recall: 0.6789 - accuracy: 0.3858 - loss: 0.6914 - val_Precision: 0.0093 - val_Recall: 1.0000 - val_accuracy: 0.0093 - val_loss: 0.7024
Epoch 3/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 609s 275ms/step - Precision: 0.0119 - Recall: 0.4126 - accuracy: 0.5700 - loss: 0.6907 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.9907 - val_loss: 0.6918
Epoch 4/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 620s 274ms/step - Precision: 0.0128 - Recall: 0.5583 - accuracy: 0.4107 - loss: 0.7181 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.9907 - val_loss: 0.6677
Epoch 5/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 624s 275ms/step - Precision: 0.0094 - Recall: 0.1417 - accuracy: 0.8358 - loss: 0.6

In [ ]:
from sklearn.metrics import roc_curve

y_pred_probs = model.predict(X_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

# Find optimal threshold (e.g., maximum TPR - FPR)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold}")

# Apply threshold to predictions
y_pred = (y_pred_probs > optimal_threshold).astype(int)

47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step
Optimal Threshold: 0.5286065340042114


In [ ]:
from sklearn.metrics import classification_report

y_true = y_test.flatten()

print("Classification Report:")
print(classification_report(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.84      0.91      1486
           1       0.02      0.36      0.04        14

    accuracy                           0.84      1500
   macro avg       0.51      0.60      0.48      1500
weighted avg       0.98      0.84      0.90      1500



In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(defog) - window_size + 1, step)
]

y_train = [
    max(defog['StartHesitation'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_train = np.array(y_train).reshape(-1, 1)

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

X_train shape: (66126, 300, 3), y_train shape: (66126, 1)


In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(defog_test) - window_size + 1, step)
]

y_test = [
    max(defog_test['StartHesitation'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_test = np.array(y_test).reshape(-1, 1)

# Verify shapes
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_test shape: (1500, 300, 3), y_test shape: (1500, 1)


In [ ]:
#training an LSTM model on defog - start hesitation
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=32,
    class_weight=class_weight_dict
)

# Evaluate the model
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 588s 280ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.9962 - loss: 2.0182 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.4866
Epoch 2/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 634s 286ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.9923 - loss: 0.7069 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.3498
Epoch 3/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 619s 284ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.9619 - loss: 0.9492 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.5482
Epoch 4/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 612s 280ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.9067 - loss: 0.8303 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.5153
Epoch 5/5
2067/2067 ━━━━━━━━━━━━━━━━━━━━ 625s 281ms/step - Precision: 3.4379

In [ ]:
from sklearn.metrics import roc_curve

y_pred_probs = model.predict(X_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

# Find optimal threshold (e.g., maximum TPR - FPR)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold}")

# Apply threshold to predictions
y_pred = (y_pred_probs > optimal_threshold).astype(int)

47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step
Optimal Threshold: inf


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report

y_true = y_test.flatten()

print("Classification Report:")
print(classification_report(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1500

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [ ]:
#TDCSFOG

In [ ]:
# Parameters
window_size = 384
overlap = 128

# Create overlapping windows for training and test data
X_train = create_overlapping_windows(tdcsfog, window_size, overlap)
X_test = create_overlapping_windows(tdcsfog_test, window_size, overlap)

In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(tdcsfog) - window_size + 1, step)
]

y_train = [
    max(tdcsfog['Turn'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_train = np.array(y_train).reshape(-1, 1)

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

X_train shape: (27405, 384, 3), y_train shape: (27405, 1)


In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(tdcsfog_test) - window_size + 1, step)
]

y_test = [
    max(tdcsfog_test['Turn'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_test = np.array(y_test).reshape(-1, 1)

# Verify shapes
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_test shape: (182, 384, 3), y_test shape: (182, 1)


In [ ]:
# Count label occurrences in y_train
unique, counts = np.unique(y_train, return_counts=True)
print("Training Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

# Count label occurrences in y_test
unique, counts = np.unique(y_test, return_counts=True)
print("\nTest Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

Training Label Distribution:
Label 0: 19680 samples
Label 1: 7725 samples

Test Label Distribution:
Label 0: 68 samples
Label 1: 114 samples


In [ ]:
#training an LSTM model on tdcsfog turn
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=16,
    class_weight=class_weight_dict,
)

# Evaluate the model
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 563s 324ms/step - Precision: 0.3626 - Recall: 0.5524 - accuracy: 0.6027 - loss: 0.6631 - val_Precision: 0.6739 - val_Recall: 0.5439 - val_accuracy: 0.5495 - val_loss: 0.6908
Epoch 2/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 563s 325ms/step - Precision: 0.3282 - Recall: 0.5618 - accuracy: 0.5530 - loss: 0.6773 - val_Precision: 0.6298 - val_Recall: 1.0000 - val_accuracy: 0.6319 - val_loss: 0.6785
Epoch 3/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 554s 321ms/step - Precision: 0.2919 - Recall: 0.5847 - accuracy: 0.4862 - loss: 0.6904 - val_Precision: 0.6264 - val_Recall: 1.0000 - val_accuracy: 0.6264 - val_loss: 0.6840
Epoch 4/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 544s 310ms/step - Precision: 0.2934 - Recall: 0.6437 - accuracy: 0.4531 - loss: 0.6966 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.3736 - val_loss: 0.7010
Epoch 5/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 560s 309ms/step - Precision: 0.2685 - Recall: 0.3618 - accuracy: 0.5577 - loss: 0.6906 - va

In [ ]:
from sklearn.metrics import roc_curve

y_pred_probs = model.predict(X_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

# Find optimal threshold (e.g., maximum TPR - FPR)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold}")

# Apply threshold to predictions
y_pred = (y_pred_probs > optimal_threshold).astype(int)

6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 773ms/step
Optimal Threshold: 0.5060513019561768


In [ ]:
y_true = y_test.flatten()

print("Classification Report:")
print(classification_report(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.39      0.90      0.54        68
           1       0.71      0.15      0.25       114

    accuracy                           0.43       182
   macro avg       0.55      0.52      0.39       182
weighted avg       0.59      0.43      0.36       182



In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(tdcsfog) - window_size + 1, step)
]

y_train = [
    max(tdcsfog['Walking'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_train = np.array(y_train).reshape(-1, 1)

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

X_train shape: (27405, 384, 3), y_train shape: (27405, 1)


In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(tdcsfog_test) - window_size + 1, step)
]

y_test = [
    max(tdcsfog_test['Walking'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_test = np.array(y_test).reshape(-1, 1)

# Verify shapes
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_test shape: (182, 384, 3), y_test shape: (182, 1)


In [ ]:
# Count label occurrences in y_train
unique, counts = np.unique(y_train, return_counts=True)
print("Training Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

# Count label occurrences in y_test
unique, counts = np.unique(y_test, return_counts=True)
print("\nTest Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

Training Label Distribution:
Label 0: 26454 samples
Label 1: 951 samples

Test Label Distribution:
Label 0: 164 samples
Label 1: 18 samples


In [ ]:
#training an LSTM model on tdcsfog walking
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=16,
    class_weight=class_weight_dict,
)

# Evaluate the model
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 570s 331ms/step - Precision: 0.0376 - Recall: 0.4329 - accuracy: 0.6146 - loss: 0.6853 - val_Precision: 0.1000 - val_Recall: 1.0000 - val_accuracy: 0.1099 - val_loss: 0.7447
Epoch 2/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 586s 310ms/step - Precision: 0.0391 - Recall: 0.7622 - accuracy: 0.3117 - loss: 0.7037 - val_Precision: 0.0989 - val_Recall: 1.0000 - val_accuracy: 0.0989 - val_loss: 0.7232
Epoch 3/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 555s 306ms/step - Precision: 0.0347 - Recall: 0.7177 - accuracy: 0.2762 - loss: 0.7071 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.9011 - val_loss: 0.5842
Epoch 4/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 561s 305ms/step - Precision: 0.0461 - Recall: 0.5039 - accuracy: 0.5329 - loss: 0.6836 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.8681 - val_loss: 0.6903
Epoch 5/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 557s 302ms/step - Precision: 0.0327 - Recall: 0.4580 - accuracy: 0.5274 - loss: 0.6

In [ ]:
from sklearn.metrics import roc_curve

y_pred_probs = model.predict(X_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

# Find optimal threshold (e.g., maximum TPR - FPR)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold}")

# Apply threshold to predictions
y_pred = (y_pred_probs > optimal_threshold).astype(int)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 207ms/step
Optimal Threshold: 0.43967127799987793


In [ ]:
from sklearn.metrics import classification_report
y_true = y_test.flatten()

print("Classification Report:")
print(classification_report(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.91      0.91       164
           1       0.07      0.06      0.06        18

    accuracy                           0.83       182
   macro avg       0.48      0.49      0.48       182
weighted avg       0.82      0.83      0.82       182



In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(tdcsfog) - window_size + 1, step)
]

y_train = [
    max(tdcsfog['StartHesitation'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_train = np.array(y_train).reshape(-1, 1)

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

X_train shape: (27405, 384, 3), y_train shape: (27405, 1)


In [ ]:
# Generate window indices
step = window_size - overlap
windows_indices = [
    (start, start + window_size)
    for start in range(0, len(tdcsfog_test) - window_size + 1, step)
]

y_test = [
    max(tdcsfog_test['StartHesitation'][start:end])
    for start, end in windows_indices
]

# Convert y_train to a numpy array
y_test = np.array(y_test).reshape(-1, 1)

# Verify shapes
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_test shape: (182, 384, 3), y_test shape: (182, 1)


In [ ]:
# Count label occurrences in y_train
unique, counts = np.unique(y_train, return_counts=True)
print("Training Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

# Count label occurrences in y_test
unique, counts = np.unique(y_test, return_counts=True)
print("\nTest Label Distribution:")
for label, count in zip(unique, counts):
    print(f"Label {label}: {count} samples")

Training Label Distribution:
Label 0: 26118 samples
Label 1: 1287 samples

Test Label Distribution:
Label 0: 136 samples
Label 1: 46 samples


In [ ]:
#training an LSTM model on tdcsfog sh
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=16,
    class_weight=class_weight_dict,
)

# Evaluate the model
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 538s 311ms/step - Precision: 0.0591 - Recall: 0.6023 - accuracy: 0.5271 - loss: 0.6811 - val_Precision: 0.2552 - val_Recall: 0.8043 - val_accuracy: 0.3571 - val_loss: 0.9868
Epoch 2/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 521s 304ms/step - Precision: 0.1294 - Recall: 0.5839 - accuracy: 0.7963 - loss: 0.5969 - val_Precision: 0.1895 - val_Recall: 0.3913 - val_accuracy: 0.4231 - val_loss: 1.0025
Epoch 3/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 565s 306ms/step - Precision: 0.0862 - Recall: 0.7695 - accuracy: 0.5870 - loss: 0.6262 - val_Precision: 0.2481 - val_Recall: 0.7174 - val_accuracy: 0.3791 - val_loss: 1.0188
Epoch 4/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 557s 303ms/step - Precision: 0.1343 - Recall: 0.6713 - accuracy: 0.7759 - loss: 0.5688 - val_Precision: 0.2597 - val_Recall: 0.8696 - val_accuracy: 0.3407 - val_loss: 0.9875
Epoch 5/5
1713/1713 ━━━━━━━━━━━━━━━━━━━━ 520s 303ms/step - Precision: 0.0731 - Recall: 0.6516 - accuracy: 0.5714 - loss: 0.6482 - val_Precis

In [ ]:
from sklearn.metrics import roc_curve

y_pred_probs = model.predict(X_test).flatten()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

# Find optimal threshold (e.g., maximum TPR - FPR)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold}")

# Apply threshold to predictions
y_pred = (y_pred_probs > optimal_threshold).astype(int)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step
Optimal Threshold: 0.5200673341751099


In [ ]:
from sklearn.metrics import classification_report
y_true = y_test.flatten()

print("Classification Report:")
print(classification_report(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.26      0.40       136
           1       0.30      0.93      0.45        46

    accuracy                           0.43       182
   macro avg       0.61      0.60      0.43       182
weighted avg       0.76      0.43      0.42       182

